# Flux Balance Analysis

## FBA

FreeFlux is capable of solving the canonical [flux balance analysis](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3108565/) (FBA) problem, which is defined as:

\begin{aligned}
\max \quad & {{\bf{c}}^T} {\bf{v}}\\
s.t. \quad & {\bf{S}} {\bf{v}} = {\bf{0}}\\
&{{\bf{v}}_{lb}} \le {\bf{v}} \le {{\bf{v}}_{ub}}\\
\end{aligned}

where ${\bf{S}}$ is the stoichiometric matrix, and ${\bf{v}}$ is the flux vector bounded by ${{\bf{v}}_{lb}}$ and ${{\bf{v}}_{ub}}$. The objective function could be any linear combination of fluxes, whereas usually biomass formation is used.

Here we use the *E. coli* metabolic network for example. The model file can be found [here](https://github.com/Chaowu88/freeflux/tree/main/models/ecoli).

In [6]:
from freeflux import Model

MODEL_FILE = 'ecoli_reactions.xlsx' # 'path/to/reactions.xlsx'

model = Model('ecoli')
model.read_from_file(MODEL_FILE)

with model.optimizer() as opt:
    # set bounds for fluxes
    opt.set_flux_bounds('all', bounds = [-100, 100]) 
    opt.set_flux_bounds('glk', bounds = [10, 10])

    # solve the FBA problem
    opt.prepare()
    res = opt.optimize(objective = {'biom': 1})

print('objective:', res.opt_objective)

glycolysis_enzymes = ['pgi', 'pfk','fba', 'tpi', 'gapdh', 'eno', 'pk']
for enzyme in glycolysis_enzymes:
    print(enzyme, round(res.opt_fluxes[enzyme], 3))


optimizing [elapsed: 0:00:01]

objective: 1.237
pgi 9.746
pfk 8.769
fba 8.769
tpi 8.769
gapdh 16.778
eno 14.625
pk 10.808


## FVA

[Flux variable analysis](https://pubmed.ncbi.nlm.nih.gov/20920235/) (FVA) is used to find the minimum and maximum of a flux while maintaining the objective above some fraction of its optimized value in the original FBA problem, which is formulated as: 

\begin{aligned}
\max/min \quad & {\rm{ }}{v_i}\\
s.t. \quad & {\bf{S}} {\bf{v}} = {\bf{0}}\\
&{{\bf{c}}^T} {\bf{v}} \ge \gamma  \cdot ob{j_{FBA}}\\
&{{\bf{v}}_{lb}} \le {\bf{v}} \le {{\bf{v}}_{ub}}\\
\end{aligned}

where $\gamma$ controls how close the FBA objective should be to its optimal value $ob{j_{FBA}}$. 

FreeFlux contains the function for FVA because it can provide a reasonable feasible region for fluxes and initial guesses sampled from it during the least squares optimzation. As an example, the flux ranges of the *E. coli* model can be estimated using the following lines:

In [7]:
with model.optimizer() as opt:
    # set bounds for fluxes
    opt.set_flux_bounds('all', bounds = [-100, 100]) 
    opt.set_flux_bounds('glk', bounds = [10, 10])

    # estimate flux ranges
    opt.prepare()
    res = opt.estimate_fluxes_range(objective = {'biom': 1}, gamma = 0)

for enzyme in glycolysis_enzymes:
    print(enzyme, [round(res.flux_ranges[enzyme][0], 3), round(res.flux_ranges[enzyme][1], 3)])


estimating flux ranges [elapsed: 0:00:12]

pgi [-18.946, 10.0]
pfk [0.0, 10.0]
fba [0.0, 10.0]
tpi [0.0, 10.0]
gapdh [8.055, 20.0]
eno [5.951, 20.0]
pk [0.0, 26.42]
